In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data = pd.read_parquet('codedData.parquet')

In [8]:
#group data by s_residence and sum weight_peak
print(data.groupby(['s_hisp']).sum()['weight_peak'].reset_index())
# calculate the percentage of each group dont include nulls
round(data.groupby(['s_hisp']).sum()['weight_peak'].reset_index()['weight_peak']/data.groupby(['s_hisp']).sum()['weight_peak'].reset_index()['weight_peak'].sum(), 2)

  s_hisp  weight_peak
0     No       251.66
1    Yes         9.60


0    0.96
1    0.04
Name: weight_peak, dtype: float64

In [13]:
(983.26)/(58.10+983.26)

0.9442075747099947